## Data Wrangling with Pandas

**[Pandas](http://pandas.pydata.org)** is a Python library that provides extensive means for data analysis. Data scientists often work with data stored in table formats like `.csv`, `.tsv`, or `.xlsx`. Pandas makes it very convenient to load, process, and analyze such tabular data using SQL-like queries. In conjunction with `Matplotlib` and `Seaborn`, `Pandas` provides a wide range of opportunities for visual analysis of tabular data. 

The main data structures in `Pandas` are implemented with **Series** and **DataFrame** classes. The former is a one-dimensional indexed array of some fixed data type. The latter is a two-dimensional data structure - a table - where each column contains data of the same type. You can see it as a dictionary of `Series` instances. `DataFrames` are great for representing real data: rows correspond to instances (examples, observations, etc.), and columns correspond to features of these instances.

## Books: https://github.com/iamseancheney/pythonbooks

---
## Data: Climate Change Laws of the World

**Let's get the data**  
http://www.lse.ac.uk/GranthamInstitute/climate-change-laws-of-the-world/

```
Selected results from the 2016 Global Climate Legislation Database

Created: 08 Dec 2019
Source: http://www.lse.ac.uk//GranthamInstitute/climate-change-laws-of-the-world/?region=all&country=all&fromyear=all&toyear=all&emitter=all&income=all&framework=all&execleg=all&category=all&document=all&type=law

Data file: data.csv
Data descriptor: datapackage.json
Data descriptor (alt): info.txt
Data Fields: Country, Name, Year Passed, Executive/Legislative, Framework, Categories, Document Type

Terms of use: This authors encourage the use of this legislation database. Users are welcome to download, save, or distribute the results electronically or in any other format, without written permission of the authors. Please reference the source as the Grantham Research Institute, London School of Economics, Global Climate Legislation database, http://www.lse.ac.uk/GranthamInstitute/Legislation/

```

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('law_search/data.csv')

**Shape of the data**

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.columns

In [ ]:
data.info()

In order to see statistics on non-numerical features, one has to explicitly indicate data types of interest in the `include` parameter.

In [ ]:
data.describe(include=['object'])

For categorical (type `object`) and boolean (type `bool`) features we can use the `value_counts` method. Let’s have a look at the distribution of `Sector`:

To calculate fractions, pass `normalize=True` to the `value_counts` function.

In [ ]:
data['Document Type'].value_counts(normalize=True)

In [ ]:
data['Executive/Legislative'].unique()

### Preliminary exploration

In [ ]:
table = \
data.groupby(['Country','Year Passed','Executive/Legislative'])\
    .count()\
    .reset_index()\
    .rename(columns = {'Year Passed':'Year','Executive/Legislative':'E/L','Name':'Acts'})\
    .sort_values('Acts', ascending=False)\
    .reset_index(drop=True)[['Country','Year','E/L','Acts']]\

In [ ]:
#table.to_csv('my_table.csv')

In [ ]:
table.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.bar(table['E/L'], table['Acts'])

In [ ]:
plt.barh(table.Year, table.Acts)

**Uniqueness**

In [ ]:
countries = data.Country.unique()

In [ ]:
print(len(countries), countries)

**Missing values**

In [ ]:
mis_val = data.isnull().sum()
mis_val_percent = 100 * data.isnull().sum()/len(data)
mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

In [ ]:
data.isnull().sum()

In [ ]:
data[data['Document Type'].isnull()]

**Direct replacement**

In [ ]:
data.loc[[1391],'Document Type'] = 'Law'

In [ ]:
#data[data['Framework'].isnull()]

**Loop replacement**

In [ ]:
indexes_to_check = data[data['Framework'].isnull()].index

In [ ]:
for i in indexes_to_check:
    data.loc[i,'Framework'] = 'Not defined'

**Just Fill in**

In [ ]:
data.Framework.fillna(value=0, inplace=True)
#data.fillna(value=0, inplace=True)


**Or just don't take the na events into account**

In [ ]:
data.dropna(inplace=True)

Let's take a look at the data

In [ ]:
data.head(10)

### Data update

Let's suppose we want to change the semicolon for pipes

In [ ]:
for i in data.index:
    data.loc[i,'Categories'] = data.Categories.loc[i].replace(';','|')

In [ ]:
data.Framework.unique()

### Data Transformations

Supose it is needed a subset of the dataset

In [ ]:
data['Executive/Legislative'].unique()

In [ ]:
data['Year Passed'].unique()

**Slicing with logical operations**

In [ ]:
law_df = data[(data['Document Type'] == 'Law') & (data['Year Passed'] >= 2000)]

**Columns selection**

In [ ]:
law_df.columns

**Explicit**

In [ ]:
law_df = law_df[['Country','Name','Year Passed']] #,'Document Type'

**Exception**

In [ ]:
law_df.iloc[:,~law_df.columns.isin(['Name'])].head()

**Pivoting**

In [ ]:
law_df.head()

In [ ]:
summary_1 = pd.pivot_table(law_df, index='Country', columns='Year Passed',aggfunc='count', fill_value=0)

In [ ]:
summary_1.head()

**Transposing**

In [ ]:
transposed = summary_1.T

In [ ]:
transposed.head()

**Write to file**

In [ ]:
transposed.to_csv('name_of_file.csv')

In [ ]:
!rm name_of_file.csv

In [ ]:
excelbook = pd.ExcelWriter('output.xlsx')

In [ ]:
summary_1.to_excel(excelbook, 'sheet1')
transposed.to_excel(excelbook, 'sheet2')
excelbook.save()

In [ ]:
!rm output.xlsx

### Enriching the dataset

We can always extend the knowledge of the dataset, creating new variables or gathering from external sources

**Conditional labeling**

In [ ]:
table.head()

In [ ]:
acts_label = []
for i in table.Acts:
    if i >= 5:
        acts_label.append('>5')
    else:
        acts_label.append('<5')

In [ ]:
table['acts_label'] = acts_label

In [ ]:
table.head()

**Retrieving links**  
Elaborate on :https://www.pingshiuanchua.com/blog/post/scraping-search-results-from-google-search

In [ ]:
data.head()

In [ ]:
import urllib
from bs4 import BeautifulSoup
import requests

In [ ]:
text = 'Climate Act Netherlands'
number_result = 1
text = urllib.parse.quote_plus(text)
url = 'https://google.com/search?q=' + text+'&num=' + str(number_result)

In [ ]:
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})

In [ ]:
links = []
for r in result_div:
    link = r.find('a', href = True)
    links.append(link['href'])

In [ ]:
links